In [30]:
import os, mlflow


In [31]:
%pwd

'/Users/vivekrauniyar/Documents/DS Stuffs/Python_DS/Projects/Kidney-Disease-Classification-project'

In [17]:
os.chdir("../")

In [32]:
%pwd

'/Users/vivekrauniyar/Documents/DS Stuffs/Python_DS/Projects/Kidney-Disease-Classification-project'

In [33]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/VivekRauniyar18/Kidney-Disease-Classification-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="VivekRauniyar18"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3e56bc451f33247f3fc88ff9070d4dd17ffde709"

In [34]:
import tensorflow as tf

In [35]:
from pathlib import Path
Path("artifacts/training/model.h5").exists()


True

In [36]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    experiment_name: str
    run_name: str
    mlflow_uri: str


In [38]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [39]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir),
            mlflow_uri=self.config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            experiment_name="kidney-disease-experiment",
            run_name="run-1"
        )
        return eval_config


In [40]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [41]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"

        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            shuffle = False,
            subset = "validation",
            
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data=scores)

    

    def log_into_mlflow(self):
        import os
        import shutil
        from pathlib import Path
        import mlflow
        from mlflow.models import infer_signature

        # --- 1) Point MLflow to DagsHub in-code (works even if env vars aren't loaded) ---
        DAGSHUB_USER = "VivekRauniyar18"
        DAGSHUB_REPO = "Kidney-Disease-Classification-Deep-Learning-Project"  # <- double-check repo name
        MLFLOW_URI = f"https://dagshub.com/{DAGSHUB_USER}/{DAGSHUB_REPO}.mlflow"

        os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_URI
        os.environ.setdefault("MLFLOW_TRACKING_USERNAME", DAGSHUB_USER)
        # If you prefer a token:
        # os.environ.setdefault("MLFLOW_TRACKING_PASSWORD", "<your_token_here>")
        # (If you already export these in the shell, this won't overwrite them.)

        mlflow.set_tracking_uri(MLFLOW_URI)
        mlflow.set_registry_uri(MLFLOW_URI)  # harmless; registry ops will be skipped

        # Optional: organize runs
        mlflow.set_experiment(getattr(self.config, "mlflow_experiment_name", "kidney-eval"))

        # --- 2) Start run and log params/metrics ---
        with mlflow.start_run() as run:
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": float(self.score[0]), "accuracy": float(self.score[1])})

            # --- 3) Add a signature to silence the warning ---
            # grab a small batch from the validation generator
            x_batch, _ = next(self.valid_generator)
            preds = self.model.predict(x_batch, verbose=0)
            signature = infer_signature(x_batch, preds)
            input_example = x_batch[:2]  # tiny example

            # --- 4) Save as TF SavedModel and log as artifacts (works with Keras 3) ---
            export_dir = Path("tmp_saved_model")
            if export_dir.exists():
                shutil.rmtree(export_dir)
            self.model.save(export_dir.as_posix())  # Keras SavedModel

            # Log the folder (generic artifacts)
            mlflow.log_artifacts(export_dir.as_posix(), artifact_path="model")

            # Also log via the TensorFlow flavor WITHOUT registry (handles signature)
            try:
                import mlflow.tensorflow as mlt
                mlt.log_model(
                    self.model,
                    artifact_path="tf_flavor_model",
                    signature=signature,
                    input_example=input_example,
                    # Do NOT pass registered_model_name on DagsHub (registry not supported)
                )
            except Exception as e:
                print("[MLflow] TF flavor log_model skipped:", e)

            print("[MLflow] tracking_uri:", mlflow.get_tracking_uri())
            print("[MLflow] run_id:", run.info.run_id)


In [42]:
import os, mlflow
print("URI env:", os.environ.get("MLFLOW_TRACKING_URI"))
print("URI api:", mlflow.get_tracking_uri())


URI env: https://dagshub.com/VivekRauniyar18/Kidney-Disease-Classification-Project.mlflow
URI api: https://dagshub.com/VivekRauniyar18/Kidney-Disease-Classification-Deep-Learning-Project.mlflow


In [45]:
from pathlib import Path
from PIL import Image, UnidentifiedImageError
import os

root = Path("artifacts/data_ingestion")  # change if needed
bad = []
for p in root.rglob("*"):
    if p.is_file() and p.suffix.lower() in {".jpg", ".jpeg", ".png", ".bmp", ".ppm", ".tif", ".tiff"}:
        try:
            with Image.open(p) as im:
                im.verify()  # quick check
        except (UnidentifiedImageError, OSError):
            bad.append(str(p))

print("Bad files:", len(bad))
for b in bad[:50]:  # print a sample
    print(b)

# If you want to delete them:
# for b in bad:
#     os.remove(b)



Bad files: 0


In [46]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e


Reading YAML from path: config/config.yaml
[2025-08-13 10:27:54,229: INFO: common: yaml file: config/config.yaml loaded successfully]
Reading YAML from path: params.yaml
[2025-08-13 10:27:54,231: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 10:27:54,232: INFO: common: created directory at: artifacts]
Found 2208 images belonging to 2 classes.
138/138 [==============================] - 138s 999ms/step - loss: 0.0000e+00 - accuracy: 1.0000
[2025-08-13 10:30:12,574: INFO: common: JSON file saved successfully at: scores.json]
INFO:tensorflow:Assets written to: tmp_saved_model/assets
[2025-08-13 10:30:14,950: INFO: builder_impl: Assets written to: tmp_saved_model/assets]
INFO:tensorflow:Assets written to: /var/folders/qb/y15c8p3j2v5122w7hy2sn6g80000gn/T/tmpds3tm22a/model/data/model/assets
[2025-08-13 10:30:29,987: INFO: builder_impl: Assets written to: /var/folders/qb/y15c8p3j2v5122w7hy2sn6g80000gn/T/tmpds3tm22a/model/data/model/assets]
[MLflow] TF flavor log_model s